In [ ]:
# default_exp pipetography

# Pipetography

> Nipype and mrtrix3 based pre-/post- processing pipeline for brain diffusion-MRI and generation of structural connectomes of the brain.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import os, sys

from nipype.interfaces.base import CommandLine, CommandLineInputSpec, File
from nipype.interfaces.base import TraitedSpec, traits
from nipype.interfaces.io import SelectFiles, DataSink, BIDSDataGrabber
from bids.layout import BIDSLayout

Environment testing before we move on to the actual code:

In [ ]:
# test if our sample data can create BIDS layouts
data_dir = 'data'

## test datasets first:
data1 = os.path.join(data_dir, 'diffantom-bids/')
data2 = os.path.join(data_dir, 'healthy_control/')

layout = BIDSLayout(data1)
assert len(layout.get_subjects()) != 0
assert type(layout.get_subjects() is list)

layout = BIDSLayout(data2)
assert len(layout.get_subjects()) != 0
assert type(layout.get_subjects() is list)

In [ ]:
# test if FSL is set up properly
assert "FSLOUTPUTTYPE" in os.environ
assert "FSLDIR" in os.environ
if "FSLOUTPUTTYPE" in os.environ:
    pass
else:
    sys.exit('FSLOUTPUTTYPE is not defined, make sure FSL is configured!')

if "FSLDIR" in os.environ:
    pass
else:
    sys.exit('FSLOUTPUTTYPE is not defined, make sure FSL is configured!')

First function to grab bvals and bvecs as fslgradient inputs to mrtrix functions

In [ ]:
#export
def bfiles2tuple(in_List):
    """
    read .bvec and .bval files in as a list and spit out tuple for nipype input
    """
    bvs_tuple = tuple(in_List)[::-1]
    return bvs_tuple

In [ ]:
# example usage:

subs = layout.get_subjects()
dwi = layout.get(subject = subs, suffix = 'dwi', extensions = ['.nii', '.nii.gz'], return_type = 'file')
bvs = layout.get(subject = subs, suffix = 'dwi', extensions = ['bvec', 'bval'], return_type = 'file')
print(dwi)
print(len(bvs))
bfiles_out = bfiles2tuple(bvs)
print(bfiles_out)

In [ ]:
# test if bfiles are grabbed
assert len(bfiles2tuple(bvs)) == 2

In [ ]:
#export
def mask2seedtuple(mask_file, grid_size):
    """
    convert mask file into appropriate input tuple for nipype node
    """
    seed_grid_tuple = (mask_file, grid_size,)
    return seed_grid_tuple